Graph Neural Networks (GNN) - Tổng quan
Định nghĩa
GNN là một loại mạng nơron sâu được thiết kế để xử lý dữ liệu có cấu trúc đồ thị (graph), trong đó các nút đại diện cho thực thể và các cạnh biểu diễn mối quan hệ giữa chúng.

1. Điểm mạnh

Xử lý dữ liệu đồ thị: Có khả năng học biểu diễn của nút, cạnh và toàn bộ đồ thị.

Khả năng tổng hợp thông tin: Trao đổi thông tin giữa các nút qua các tầng mạng.

Ứng dụng rộng rãi: Hỗ trợ nhiều lĩnh vực có dữ liệu phức tạp.

2. Điểm yếu

Khả năng mở rộng: Khó xử lý đồ thị lớn với nhiều nút và cạnh.

Over-smoothing: Trạng thái của các nút có thể trở nên đồng nhất sau nhiều lớp GNN.

Tính toán phức tạp: Yêu cầu tài nguyên lớn để huấn luyện trên đồ thị lớn.

3.
Ứng dụng

Mạng xã hội: Dự đoán mối quan hệ, phát hiện cộng đồng.

Hệ thống khuyến nghị: Đề xuất sản phẩm dựa trên quan hệ giữa người dùng và sản phẩm.

Sinh học: Phân tích cấu trúc protein, dự đoán tương tác phân tử.

Giao thông: Tối ưu hóa luồng giao thông và định tuyến.

4. Tóm tắt:

Mạng Nơ-ron Đồ Thị (GNN) là mô hình học máy được thiết kế để xử lý dữ liệu đồ thị, nơi các đỉnh và cạnh biểu diễn đối tượng và mối quan hệ. GNN truyền và kết hợp thông tin từ các đỉnh láng giềng để học đặc trưng và cấu trúc đồ thị, giúp nắm bắt các mối quan hệ phức tạp. Mô hình này hiệu quả trong các nhiệm vụ như phân loại nút, phân loại đồ thị, dự đoán liên kết và nhúng đồ thị. GNN có nhiều ứng dụng, từ phân tích mạng xã hội, hệ thống gợi ý đến phát hiện các hiện tượng trong hệ thống phức tạp.

In [ ]:
# Tiếp theo là dùng GCN

In [2]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.7 MB/s eta 0:00:00


# Dữ liệu được sử dụng karate club

In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import KarateClub
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges, negative_sampling
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Tải dataset Karate Club
dataset = KarateClub()
data = dataset[0]

# Tạo dữ liệu cho bài toán dự đoán liên kết
data = train_test_split_edges(data)  # Tự động tạo tập train, val, test cho các liên kết

# Tạo các liên kết âm cho tập huấn luyện
data.train_neg_edge_index = negative_sampling(
    edge_index=data.train_pos_edge_index,
    num_nodes=data.num_nodes
)

# Xây dựng mô hình GCN
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)  # Lớp GCN đầu tiên
        self.conv2 = GCNConv(16, out_channels)  # Lớp GCN thứ hai

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)  # Tích chập đầu tiên
        x = F.relu(x)                  # Hàm kích hoạt ReLU
        x = self.conv2(x, edge_index)  # Tích chập thứ hai
        return x

# Hàm dự đoán liên kết (dot product giữa các nút)
def link_prediction(z, edge_label_index):
    return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=1)

# Khởi tạo mô hình và tham số
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(data.num_features, 16).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Huấn luyện mô hình
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    z = model(data.x, data.train_pos_edge_index)  # Biểu diễn nút học được
    pos_pred = link_prediction(z, data.train_pos_edge_index)  # Dự đoán liên kết dương
    neg_pred = link_prediction(z, data.train_neg_edge_index)  # Dự đoán liên kết âm

    # Tính loss
    loss = F.binary_cross_entropy_with_logits(
        torch.cat([pos_pred, neg_pred]),
        torch.cat([torch.ones(pos_pred.size(0)), torch.zeros(neg_pred.size(0))]).to(device)
    )
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Đánh giá mô hình
model.eval()
with torch.no_grad():
    z = model(data.x, data.train_pos_edge_index)
    pos_pred = torch.sigmoid(link_prediction(z, data.test_pos_edge_index))
    neg_pred = torch.sigmoid(link_prediction(z, data.test_neg_edge_index))

    # Tính các chỉ số đánh giá
    y_true = torch.cat([torch.ones(pos_pred.size(0)), torch.zeros(neg_pred.size(0))]).cpu().numpy()
    y_pred = torch.cat([pos_pred, neg_pred]).cpu().numpy()

    # Tính độ chính xác
    acc = accuracy_score(y_true, (y_pred > 0.5).astype(int))

    # Tính độ chính xác, recall, F1-score
    precision = precision_score(y_true, (y_pred > 0.5).astype(int))
    recall = recall_score(y_true, (y_pred > 0.5).astype(int))
    f1 = f1_score(y_true, (y_pred > 0.5).astype(int))

    # Tính AUC-ROC
    auc = roc_auc_score(y_true, y_pred)

    print(f"Độ chính xác: {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"AUC-ROC: {auc:.4f}")


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Epoch 0, Loss: 0.6901
Epoch 20, Loss: 0.5080
Epoch 40, Loss: 0.3968
Epoch 60, Loss: 0.3621
Epoch 80, Loss: 0.3277
Epoch 100, Loss: 0.2843
Epoch 120, Loss: 0.2343
Epoch 140, Loss: 0.1856
Epoch 160, Loss: 0.1375
Epoch 180, Loss: 0.1023
Độ chính xác: 0.6429
Precision: 0.6667
Recall: 0.5714
F1-score: 0.6154
AUC-ROC: 0.6531


Method       AUC  Accuracy  Precision    Recall  F1-score
0         Common Neighbors  0.493333  0.400000   0.400000  0.400000  0.400000
1      Jaccard Coefficient  0.422222  0.400000   0.400000  0.400000  0.400000
2              Adamic-Adar  0.515556  0.400000   0.400000  0.400000  0.400000
3  Preferential Attachment  0.702222  0.466667   0.481481  0.866667  0.619048
4            Random Forest  0.777778  0.555556   0.400000  0.666667  0.500000




Bảng ở Lab4_02

<!-- Method       AUC  Accuracy  Precision    Recall  F1-score
0         Common Neighbors  0.493333  0.400000   0.400000  0.400000  0.400000
1      Jaccard Coefficient  0.422222  0.400000   0.400000  0.400000  0.400000
2              Adamic-Adar  0.515556  0.400000   0.400000  0.400000  0.400000
3  Preferential Attachment  0.702222  0.466667   0.481481  0.866667  0.619048
4            Random Forest  0.777778  0.555556   0.400000  0.666667  0.500000 -->

In [6]:
# Nhìn vào kết quả của mô hình GCN và các phương pháp khác, chúng ta có thể rút ra một số nhận xét như sau:

# So với các phương pháp khác:
# AUC-ROC:

# Mô hình GCN có AUC-ROC là 0.6531, cao hơn các phương pháp Common Neighbors, Jaccard Coefficient, Adamic-Adar và Random Forest nhưng thấp hơn phương pháp Preferential Attachment (AUC-ROC = 0.7022).
# AUC-ROC là chỉ số quan trọng đo lường khả năng phân biệt giữa các lớp (liên kết dương và âm). Mặc dù GCN không đạt hiệu suất tốt nhất, nhưng kết quả của nó vẫn cho thấy khả năng phân biệt khá tốt so với các phương pháp khác.
# Độ chính xác (Accuracy):

# Mô hình GCN đạt độ chính xác 0.6429, tốt hơn các phương pháp Common Neighbors (0.4000), Jaccard Coefficient (0.4000), Adamic-Adar (0.4000) và Random Forest (0.5556). Tuy nhiên, nó không vượt qua Preferential Attachment (0.4667).
# Accuracy là chỉ số đo lường tỷ lệ đúng trên tổng số mẫu. Mô hình GCN cho thấy hiệu suất khá mạnh, đặc biệt so với các phương pháp không học máy.
# Precision và Recall:

# Precision của GCN là 0.6667, tương đối cao, cho thấy rằng khi mô hình dự đoán một liên kết là dương, nó đúng khá nhiều lần.
# Recall của GCN là 0.5714, nghĩa là mô hình có thể phát hiện khoảng 57% các liên kết dương thực tế.
# Precision và Recall của GCN đều tốt hơn các phương pháp Common Neighbors, Jaccard Coefficient, Adamic-Adar và Random Forest (trừ Preferential Attachment). Tuy nhiên, Recall thấp hơn một chút so với phương pháp Preferential Attachment (0.8667), cho thấy GCN có thể chưa phát hiện đầy đủ tất cả các liên kết dương.
# F1-score:

# F1-score của GCN là 0.6154, khá cao và cho thấy sự cân bằng tốt giữa Precision và Recall. Mô hình GCN đạt điểm F1 cao hơn các phương pháp như Common Neighbors, Jaccard Coefficient, Adamic-Adar, và Random Forest (trừ Preferential Attachment, với F1 = 0.619048).
# F1-score là chỉ số tổng hợp giữa Precision và Recall, cho thấy mô hình GCN có khả năng cân bằng tốt giữa việc giảm số lượng dự đoán sai (false positives) và việc phát hiện các liên kết dương (recall).
# Tổng kết:
# Mô hình GCN có hiệu suất tổng thể vượt trội so với hầu hết các phương pháp dựa trên các đặc trưng đồ thị như Common Neighbors, Jaccard Coefficient và Adamic-Adar.
# Nó cũng cho thấy khả năng phân biệt và dự đoán tốt hơn so với Random Forest trong bài toán dự đoán liên kết.
# Preferential Attachment vẫn là phương pháp có hiệu suất tốt nhất, đặc biệt là với AUC-ROC cao và Recall mạnh mẽ.
# Mặc dù không đạt được điểm cao nhất ở mọi chỉ số, GCN vẫn cho thấy tiềm năng mạnh mẽ khi học từ các đặc trưng của đồ thị và có thể vượt trội trong các ứng dụng lớn hơn hoặc phức tạp hơn.